In [1]:
pip install transformers


In [4]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00


In [2]:
pip install --upgrade datasets fsspec


  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

# 1. Load a public sentiment dataset (e.g. IMDb)
dataset = load_dataset("imdb")
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize
def tokenize_fn(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length')
tokenized = dataset.map(tokenize_fn, batched=True)

# 2. Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 3. Compute metrics
metric = evaluate.load('accuracy')
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# 4. Train
args = TrainingArguments(
    output_dir="./finetuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    #evaluate_during_training=True,  # legacy flag
    logging_dir="./logs",
    eval_strategy="epoch",  # <- use this
    save_strategy="epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    report_to="tensorboard",
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)
trainer.train()
# Save model
trainer.save_model("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/tmp/ipython-input-5-548994738.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.250400,0.261303,0.897040


Epoch,Training Loss,Validation Loss,Accuracy
1,0.250400,0.261303,0.897040
2,0.201800,0.242132,0.916360
3,0.094200,0.335631,0.921760


('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/vocab.txt',
 './finetuned_model/added_tokens.json',
 './finetuned_model/tokenizer.json')

In [6]:
 !pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 22.3 MB/s eta 0:00:00


In [8]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph
from langgraph.channels import LastValue
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# === Load model ===
model_path = "/content/finetuned_model/checkpoint-4689"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).eval()
id2label = {0: "negative", 1: "positive"}  # Adjust as per your classes

# === State class ===
class MyState(TypedDict):
    text: LastValue[str]
    label_id: LastValue[int]
    confidence: LastValue[float]
    label: LastValue[str]
    route_to: LastValue[str]
    action_taken: LastValue[Optional[str]]  # optional to prevent conflicts

# === Node functions ===
def inference_node(state: MyState) -> dict:
    text = state["text"]
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=-1)
    conf, label_id = torch.max(probs, dim=-1)
    return {
        "label_id": int(label_id.item()),
        "confidence": float(conf.item()),
        "label": id2label[int(label_id.item())]
    }

def check_and_route_node(state: MyState) -> dict:
    threshold = 0.8
    return {
        "route_to": "end_node" if state["confidence"] >= threshold else "fallback_node"
    }

def fallback_node(state: MyState) -> dict:
    return {"action_taken": "Asked user for clarification"}

def end_node(state: MyState) -> dict:
    return {"action_taken": "Accepted model's prediction"}

# === Graph construction ===
graph = StateGraph(MyState)
graph.add_node("inference_node", inference_node)
graph.add_node("check_and_route_node", check_and_route_node)
graph.add_node("fallback_node", fallback_node)
graph.add_node("end_node", end_node)

# === Edges ===
graph.add_edge("__start__", "inference_node")
graph.add_edge("inference_node", "check_and_route_node")
graph.add_conditional_edges(
    "check_and_route_node",
    lambda state: state["route_to"],
    {"end_node": "end_node", "fallback_node": "fallback_node"}
)

compiled = graph.compile()
result = compiled.invoke({"text": "I did not enjoy this movie at all"})
print(result)


{'text': 'I did not enjoy this movie at all', 'label_id': 0, 'confidence': 0.9984972476959229, 'label': 'negative', 'route_to': 'end_node', 'action_taken': "Accepted model's prediction"}
